In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
import mlflow
import optuna
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [3]:

#Set up the MLflow tracking server
mlflow.set_tracking_uri('http://ec2-54-173-92-7.compute-1.amazonaws.com:5000/')

In [4]:
# Set or create an experiment
mlflow.set_experiment("Exp5: ML Algos with Hyperparameter tuning")

<Experiment: artifact_location='s3://youtubecommentmlflow/149143573475036011', creation_time=1741969037519, experiment_id='149143573475036011', last_update_time=1741969037519, lifecycle_stage='active', name='Exp5: ML Algos with Hyperparameter tuning', tags={}>

In [ ]:
df = pd.read_csv('C:/test/youtube_comment_sentiment_analysis/Artifacts/youtubecomment_preprocessing.csv')
df.shape

(15833, 2)

In [6]:
# Step 1: Remap the class labels from [-1, 0, 1] to [2, 0, 1]
df['Sentiment'] = df['Sentiment'].map({-1: 2, 0: 0, 1: 1})



ngram_range = (1, 3)  # Trigram setting
max_features = 8000  # Set max_features to 8000 for TF-IDF

# Step 4: Train-test split before vectorization and resampling
X_train, X_test, y_train, y_test = train_test_split(df['Comment'], df['Sentiment'], test_size=0.2, random_state=42, stratify=df['Sentiment'])

# Step 2: Vectorization using TF-IDF, fit on training data only
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train = vectorizer.fit_transform(X_train)  # Fit on training data
X_test = vectorizer.transform(X_test)  # Transform test data

smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

In [ ]:
# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test, params, trial_number):
    with mlflow.start_run():
        # Log model type and trial number
        mlflow.set_tag("mlflow.runName", f"Trial_{trial_number}_{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Log hyperparameters
        for key, value in params.items():
            mlflow.log_param(key, value)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")

        return accuracy

# Step 6: Optuna objective function for LightGBM
def objective_lightgbm(trial):
    # Hyperparameter space to explore
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 15)
    num_leaves = trial.suggest_int('num_leaves', 20, 150)
    min_child_samples = trial.suggest_int('min_child_samples', 10, 100)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0)
    subsample = trial.suggest_float('subsample', 0.5, 1.0)
    reg_alpha = trial.suggest_float('reg_alpha', 1e-4, 10.0, log=True)  # L1 regularization
    reg_lambda = trial.suggest_float('reg_lambda', 1e-4, 10.0, log=True)  # L2 regularization

    # Log trial parameters
    params = {
        'n_estimators': n_estimators,
        'learning_rate': learning_rate,
        'max_depth': max_depth,
        'num_leaves': num_leaves,
        'min_child_samples': min_child_samples,
        'colsample_bytree': colsample_bytree,
        'subsample': subsample,
        'reg_alpha': reg_alpha,
        'reg_lambda': reg_lambda
    }

    # Create LightGBM model
    model = LGBMClassifier(n_estimators=n_estimators,
                           learning_rate=learning_rate,
                           max_depth=max_depth,
                           num_leaves=num_leaves,
                           min_child_samples=min_child_samples,
                           colsample_bytree=colsample_bytree,
                           subsample=subsample,
                           reg_alpha=reg_alpha,
                           reg_lambda=reg_lambda,
                           random_state=42)

    # Log each trial as a separate run in MLflow
    accuracy = log_mlflow("LightGBM", model, X_train, X_test, y_train, y_test, params, trial.number)

    return accuracy

# Step 7: Run Optuna for LightGBM, log the best model, and plot the importance of each parameter
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_lightgbm, n_trials=20)  # Increased to 20 trials

    # Get the best parameters
    best_params = study.best_params
    best_model = LGBMClassifier(n_estimators=best_params['n_estimators'],
                                learning_rate=best_params['learning_rate'],
                                max_depth=best_params['max_depth'],
                                num_leaves=best_params['num_leaves'],
                                min_child_samples=best_params['min_child_samples'],
                                colsample_bytree=best_params['colsample_bytree'],
                                subsample=best_params['subsample'],
                                reg_alpha=best_params['reg_alpha'],
                                reg_lambda=best_params['reg_lambda'],
                                random_state=42)

    # Log the best model with MLflow and print the classification report
    log_mlflow("LightGBM", best_model, X_train, X_test, y_train, y_test, best_params, "Best")

    # Plot parameter importance
    optuna.visualization.plot_param_importances(study).show()

    # Plot optimization history
    optuna.visualization.plot_optimization_history(study).show()

In [10]:
# Run the experiment for LightGBM
run_optuna_experiment()

[I 2025-03-15 13:38:55,256] A new study created in memory with name: no-name-b98fa491-fc2d-47af-9383-a10d5416a491


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.137590 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 92392
[LightGBM] [Info] Number of data points in the train set: 24759, number of used features: 1154
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] 

2025/03/15 13:40:00 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\PARTHI~1\AppData\Local\Temp\tmp0yq8mn_e\model\model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.0.2', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback. 
2025/03/15 13:40:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Trial_0_LightGBM_SMOTE_TFIDF_Trigrams at: http://ec2-54-173-92-7.compute-1.amazonaws.com:5000/#/experiments/149143573475036011/runs/1863b811b68442fb82e13c9a1f332541
🧪 View experiment at: http://ec2-54-173-92-7.compute-1.amazonaws.com:5000/#/experiments/149143573475036011


[I 2025-03-15 13:40:23,379] Trial 0 finished with value: 0.6665614145879382 and parameters: {'n_estimators': 555, 'learning_rate': 0.012359179482890132, 'max_depth': 7, 'num_leaves': 132, 'min_child_samples': 82, 'colsample_bytree': 0.9713643465787258, 'subsample': 0.800171693937636, 'reg_alpha': 0.0003252580869693209, 'reg_lambda': 0.002317349590025096}. Best is trial 0 with value: 0.6665614145879382.


...

🏃 View run Trial_19_LightGBM_SMOTE_TFIDF_Trigrams at: http://ec2-54-173-92-7.compute-1.amazonaws.com:5000/#/experiments/149143573475036011/runs/1d7025a67b4b473790784eda65a24d60
🧪 View experiment at: http://ec2-54-173-92-7.compute-1.amazonaws.com:5000/#/experiments/149143573475036011


[I 2025-03-15 14:29:58,160] Trial 19 finished with value: 0.7328702241869277 and parameters: {'n_estimators': 876, 'learning_rate': 0.04419480157143808, 'max_depth': 12, 'num_leaves': 114, 'min_child_samples': 10, 'colsample_bytree': 0.9214918410190789, 'subsample': 0.7697974347550122, 'reg_alpha': 1.592908434831827, 'reg_lambda': 6.539486305110264}. Best is trial 11 with value: 0.747710767287654.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.004816 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 143931
[LightGBM] [Info] Number of data points in the train set: 24759, number of used features: 5270
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning]

2025/03/15 14:33:36 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\PARTHI~1\AppData\Local\Temp\tmp2hsl46j2\model\model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.0.2', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback. 
2025/03/15 14:33:36 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Trial_Best_LightGBM_SMOTE_TFIDF_Trigrams at: http://ec2-54-173-92-7.compute-1.amazonaws.com:5000/#/experiments/149143573475036011/runs/e37f4e0e35e24668a310dc91e7f164ac
🧪 View experiment at: http://ec2-54-173-92-7.compute-1.amazonaws.com:5000/#/experiments/149143573475036011
